# Hurto Motocicletas, Residencias & Lesiones Transito

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'
CODIGO_DANE_PATH = '/content/drive/MyDrive/Police-Data/dane_codes.xlsx'

## Libraries

In [10]:
import pandas as pd
import numpy as np
import os
import glob

In [11]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

## Read Files

In [12]:
folders = ["Hurto_Motocicletas","Hurto_Residencias","Lesiones_Accidentes_Transito"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [13]:
folder_read(folders)

## Hurto Motocicletas

In [14]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [15]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Hurto_Motocicletas']]
hurto_motocicletas_df = pd.concat(standardized_dataframes, ignore_index=True)

In [16]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','ESCOPOLAMINA','LLAVE MAESTRA','SIN EMPLEO DE ARMAS',
                            'CORTANTES','CORTOPUNZANTES','PUNZANTES','JERINGA']
hurto_motocicletas_df["AGRUPA_EDAD_PERSONA"] = hurto_motocicletas_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_motocicletas_df["GENERO"] = hurto_motocicletas_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_motocicletas_df["ARMAS_MEDIOS"] = hurto_motocicletas_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [17]:
dane_codes = pd.read_excel(CODIGO_DANE_PATH,sheet_name='Municipios_dane',dtype={'CODIGO_DANE':str})
def clean_data(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].apply(lambda x: '0' + x if len(x) == 9 else x)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].str[:5]
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  # Adjust upper case
  filtered_df = filtered_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)
  # Merge DANE CODE
  merged_df = filtered_df.merge(dane_codes, on='CODIGO_DANE', how='left', suffixes=('', '_csv'))
  # fill Municipio
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO_csv']
  merged_df = merged_df.drop(columns=['MUNICIPIO_csv'])
  merged_df['CAPITAL'] = merged_df['CODIGO_DANE'].apply(lambda codigo: 1 if codigo[-3:] == '001' else 0)
  # drop sufix (1), (2), (3)
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO'].str.replace(r'\s*\(\d+\)$', '', regex=True)
  # filter date
  output_df = merged_df[(merged_df['FECHA'] >= '2013-01-01') & (merged_df['FECHA'] <= '2023-12-31')]
  return output_df

In [18]:
hurto_motocicletas_filtered = clean_data(hurto_motocicletas_df)

In [19]:
hurto_motocicletas_output = hurto_motocicletas_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [20]:
hurto_motocicletas_output.to_csv(OUTPUT_PATH_FILE+"/hurto_motocicletas.csv",index=False)

## Hurto Residencias

In [21]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Hurto_Residencias']]
hurto_residencias_df = pd.concat(standardized_dataframes, ignore_index=True)

In [22]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','ESCOPOLAMINA','LLAVE MAESTRA','PALANCAS',
                             'SIN EMPLEO DE ARMAS','CORTANTES','JERINGA','LLAMADA TELEFONICA']
hurto_residencias_df["AGRUPA_EDAD_PERSONA"] = hurto_residencias_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_residencias_df["GENERO"] = hurto_residencias_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_residencias_df["ARMAS_MEDIOS"] = hurto_residencias_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [23]:
hurto_residencias_filtered = clean_data(hurto_residencias_df)

In [24]:
hurto_residencias_output = hurto_residencias_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [25]:
hurto_residencias_output.to_csv(OUTPUT_PATH_FILE+"/hurto_residencias.csv",index=False)

## Lesiones Accidentes Transito

In [26]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Lesiones_Accidentes_Transito']]
lesiones_accidentes_transito_df = pd.concat(standardized_dataframes, ignore_index=True)

In [27]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['BICICLETA','MOTO','SIN EMPLEO DE ARMAS','VEHICULO','ARMA BLANCA / CORTOPUNZANTE',
                             'CONTUNDENTES','ESCOPOLAMINA','ARMA DE FUEGO']
lesiones_accidentes_transito_df["AGRUPA_EDAD_PERSONA"] = lesiones_accidentes_transito_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
lesiones_accidentes_transito_df["GENERO"] = lesiones_accidentes_transito_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
lesiones_accidentes_transito_df["ARMAS_MEDIOS"] = lesiones_accidentes_transito_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [28]:
lesiones_accidentes_transito_filtered = clean_data(lesiones_accidentes_transito_df)

In [29]:
lesiones_accidentes_transito_output = lesiones_accidentes_transito_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [30]:
lesiones_accidentes_transito_output.to_csv(OUTPUT_PATH_FILE+"/lesiones_accidentes_transito.csv",index=False)